# Sizing assumptions

## Airloop flow

In [1]:
# assumptions for nominal supply airflow
m = 33 #kg/s
dT = 15 #K
c = 1005 #J/(kg*K)
ro = 1.17 # kg/m^3
V = m/ro
print(V)

V_cfm = V/0.00047194745
print(V_cfm)

28.205128205128208
59763.28128296531


In [2]:
V*200/0.7/V_cfm

0.13484212857142858

### Airflow efficiency ratio

[source s5 examples by stulz.de](https://www.stulz.de/de/newsroom/blog/aer-eine-neue-kennzahl-fuer-die-effizienz-von-luftbewegung-in-rechenzentren-352/), 
using specific supply air fan power slightly above example 1 (0.11 W/(m3/h)) in the assumptions below:

We assume dT of 15 deg for the airflow (also used in souce s6)

In [3]:
P_supply_fan = 11303.8 #W
V_air_supply_fan = 27.6942*3600 #m3/h
# airflow efficiency ratio, AER
AER = P_supply_fan / V_air_supply_fan
print("Resulting AER is", AER)

Resulting AER is 0.11337913514181469


## Chilled water flow

[source s1](https://www.trane.com/content/dam/Trane/Commercial/global/products-systems/education-training/engineers-newsletters/waterside-design/ADM-APN051-EN.pdf)

In [4]:
Cn = 471000 #W
Cn_ton = Cn/3500
print(Cn_ton)

# rule from the source 
V_gpm_min = 1.2 * Cn_ton
V_gpm = 1.5 * Cn_ton

V = V_gpm * 0.0000630901964
ro_w = 1000
m_chw_flow_nominal = V * ro_w
print(m_chw_flow_nominal, "; In our model we assume", 12, "kg/s")

134.57142857142858
12.735206787600003 ; In our model we assume 12 kg/s


In [5]:
m_chw_flow_nominal = 12 #kg/s
m_chw_flow_nominal_gpm = (12/1000)/0.0000630901964
print("Our model nominal evaporator flow in gpm/ton is:", m_chw_flow_nominal_gpm/Cn_ton)
print("Source provides for a 130 ton chiller a nominal evaporator flow in gpm/ton of", 1.5)

Our model nominal evaporator flow in gpm/ton is: 1.4134046113429595
Source provides for a 130 ton chiller a nominal evaporator flow in gpm/ton of 1.5


### Condenser water flow check

In [6]:
#  Paul's method based on ASHRAE Standard 550 / 590 : Tower is sized at 3.0 GPM per ton of plant capacity (3 gpm is 0.14 kg/s). See section 1.5 in this notebook - I copied over the content of the email
0.14*130, "Our nominal value is about", 20, "kg/s"

(18.200000000000003, 'Our nominal value is about', 20, 'kg/s')

## Condenser, evaporator, coil, and WSE pressure drops

### Chiller (based on a 150 ton Carrier Screw Chiller)
[source s3](https://chillerscarrier.com/assets/30xw-9pd.pdf)

In [7]:
# data chiller
# data chiller scaled down to 133 ton
# model chiller of 133 ton with modified nominal flows

# Assuming the geometry of the condenser and evaporator heat exchangers is such that the HX tubes are slightly narrower
# for a 133 ton chiller compared to a 150 ton chiller (these are shell-and-tube HX, see images). 

# As there is a rule (see source s1) where the nominal flow scales linearly with the chiller tonnage, we assume a linearly proportional to the tonnage decrease in
# both nominal evaporator and condenser flow for such equivalent to the data source chiller that has 133 tons

# Since the tubes of such equivalent chiller are assumed narrower, we will say that the 133 ton flows produce the same pressure drops as the 150 ton chiller from the data


#from the data
tonnage_data = 150
# from our model
tonnage_model = 133

# from the data
evap_dp = 33.2 #kPa
evap_flow = 17.91 #kg/s
# our model
m_chw_flow_nominal = 12 #kg/s

# from the data
cond_dp = 42.1 #kPa
cond_flow = 22.39 #kg/s
# our model
m_cw_flow_nominal = 39.7/2 #kg/s

# as in our model the 133 ton chiller has modifications to the nominal flow. As the flow velocity is directly proportional to the mass flow, and the pressure drop depends on the square of the velocity,
# our model pressure drops can be approximated as:

evap_dp_adjusted = evap_dp * (m_chw_flow_nominal/(evap_flow*(133/150)))**2
print("Adjusted nominal evaporator pressure drop:", evap_dp_adjusted, " kPa")
cond_dp_adjusted = cond_dp * (m_cw_flow_nominal/(cond_flow*(133/150)))**2
print("Adjusted nominal condenser pressure drop:", cond_dp_adjusted, " kPa")



Adjusted nominal evaporator pressure drop: 18.95783068821084  kPa
Adjusted nominal condenser pressure drop: 42.089538809766864  kPa


### Cooling tower

Based on [source s4](https://www.baltimoreaircoil.com/download_api_endpoint/1178/j120_tr_pumpingpiping_20150914.pdf) most flow resistances can be neglected and we will consider the pumping to overcome the height difference.

In [8]:
ro_w = 1000 # kJ/(kg*K)
gr = 9.81 #kg/(s2)
height = 1.5 #m, 5ft - confirmed per communication with Paul

coo_tow_dp = ro_w * gr * height
print("Tower dp is: ",coo_tow_dp/1000, "kPa")

Tower dp is:  14.715 kPa


### Cooling coil

#### Airside

Pressure resisting elements: coil, short duct and plenum: we assume 200 Pa, also corresponds well with this [source s6](https://www.trane.com/content/dam/Trane/Commercial/global/products-systems/education-training/engineers-newsletters/standards-codes/ADM-APN070-EN_06032019.pdf), assuming that the coil would have a larger cross-area but remain similar accross the flow direction. Our chosen value is also reasonable when comparing to the materials provided in [source s7](https://www.trane.com/content/dam/Trane/Commercial/global/products-systems/education-training/continuing-education-gbci-aia-pdh/HVAC-Coil-Selection-and-Optimization/APP-CMC054-EN_course%20material.pdf)


note: plenum and short duct almost negligible, assume they participate with about 5-10% in the total airloop flow resistance pressure drop

source_flow = 7000 #cfm

source_dp = 0.7 #in H2O

our_flow = 57209 cfm

our_dp = 0.803729

#### Waterside

Average from [source s6, table 1]

In [9]:
m_chw = 12

ft_H2O_avg = (11.2 + 4.7 + 11.1 + 5.2)/4
ft_H2O_avg_Pa = ft_H2O_avg * 2989.0669

# assuming the water flow pipes are laminated such that the increase in nominal capacity comes from adding more capacity in parallel 
# (see laminated piping image in the source),
# we can assume that the pressure drop remains the same as a smaller capacity is chosen for a lower flow
dp = ft_H2O_avg_Pa
dp

24061.988545

### Waterside economizer (WSE)

[Source s9](https://www.taylor-engineering.com/wp-content/uploads/2020/04/ASHRAE_Journal_-_How_to_Design_Control_Waterside_Economizers.pdf)

"The heat exchanger pressure drop on the condenser water side should be similar to that of the condensers so the flow rate will be similar when serving either the condensers or heat exchanger. On the chilled water side, pressure drop is typically limited to about 5 or 6 psi (34 or 41 kPa) to limit the chilled water pump energy impact."

In [10]:
# assume nominal pressure drop on the condenser water side equals the pressure drop through condenser
wse_dp_cond_side = cond_dp_adjusted
print("WSE pressure drop on the condenser water side:", wse_dp_cond_side)

# assume the same pressure drop on the evaporator side as on the condenser water side and adjust for the chilled water nominal flow
wse_dp_evap_side = wse_dp_cond_side * (m_chw_flow_nominal/m_cw_flow_nominal)**2
print("WSE pressure drop on the chilled water side:", wse_dp_evap_side)

WSE pressure drop on the condenser water side: 42.089538809766864
WSE pressure drop on the chilled water side: 15.382100231855864


## Cooling tower

Approach temperature [Source s10 (PG&E, leaving tower water temperature minus ambient wet bulb
temperature](https://www.pge.com/includes/docs/pdfs/mybusiness/energysavingsrebates/incentivesbyindustry/DataCenters_BestPractices.pdf) recommends 5 - 7 F, 
conservatively assuming

In [11]:
approachT = 4 #K

### Sizing the tower fan

In [12]:
# cold water temperature (entering the condenser)
T_cond_in_max = 297 #K
T_wet_bulb_max = 296 #K

T_cond_in_max_C = 297 - 273 #K
T_wet_bulb_max_C = 296 - 273 #K

print("T_cond_in_max_C:", T_cond_in_max_C)
print("T_wet_bulb_max_C:", T_wet_bulb_max_C)

T_cond_in_max_C: 24
T_wet_bulb_max_C: 23


**Paul's method based on ASHRAE Standard 550 / 590**
 
ASHRAE Standard 550 / 590, [source](https://www.trane.com/content/dam/Trane/Commercial/global/products-systems/education-training/engineers-newsletters/waterside-design/adm-apn045-en_0912.pdf) 

Tower is sized at 3.0 GPM per ton of plant capacity.  Note that some designs reduce this to 2.0 or 2.5 GPM per ton. 
Chiller efficiency is .57 KW / Ton.  Some machines are even more efficient - but it is hard to get much better 
than .5 KW / ton at design conditions.

Tower fan efficiency .046 KW / Ton

Condenser water pump .054 KW / Ton

Tower + Pump = .1 KW / Ton

In terms of overall plant energy use it is valuable to think of the overall plant running around .6 - .8 KW / Ton.  
Of that amount 85% is attributable to the chiller and roughly 15% to the tower fan and pump.  
So in terms of the overall lift done by the plant (difference between the CHW temperature and the outdoor air wet bulb) - the tower is roughly five times more efficient than the chiller.  This is due mostly to the fact that the tower is evaporating water - rather than trying to mechanically cool it.  So the more work we can have the tower do and the less the chiller needs to do the better.  This is why strategies like the use of WSE and CDW are so effective in reducing total plant efficiency.

In [13]:
# Tower fan efficiency .046 KW / Ton
print("Tower nominal fan power is thus: ", .046 * 142, "kW")

Tower nominal fan power is thus:  6.532 kW


## Piping/Fittings pressure drops

[source s11](https://www.advantageengineering.com/fyi/166/advantageFYI166.php)

6" CW at 2200 Pa/100ft

5" CHW at 2300 Pa/100ft

The piping length assumption are compiled in a spreadsheet [here](https://docs.google.com/spreadsheets/d/1XtaqeURHSJD2tsk9S-GBtri8GY4wZl62JM6LV3KOiVo/edit?usp=sharing). The assumption is that there is:

- In the base controller case: 10m of pipe in each chiller condenser and WSE condenser side, and 20m of pipe in the dedicated cooling tower loop.

- In the alternative controller case: 30m of pipe in the condenser water loop

- In both: 30m of pipe in the chilled water loop.

To account for all the fittings, including butterfly isolation valves we doubled the resistance obtained for the pipe length. Additional 6kPa are added for each modulating valve.

Please see the ChillerPlant package models for how the sizing values were applied. The piping resistances are for the most part passed as pressure drops in valves.

Note that the piping pressure differences are provided directly in the models, not in the base class.

## IT and non-IT data center loads
[source s12](https://www.apcdistributors.com/white-papers/Cooling/WP-25%20Calculating%20Total%20Cooling%20Requirements%20for%20Data%20Centers.pdf), Table 1. suggests 29% non-IT. Reducing the value to 20% non-IT assuming a 50% data center computing capacity average utilization factor

In [18]:
total = 500000 # W
it_loads = 0.8 * total
non_it_loads = (total - it_loads)

print(it_loads)
print(non_it_loads)

400000.0
100000.0
